In [2]:
# !pip install factor_analyzer

# Import required libraries
import pandas as pd
from sklearn.datasets import load_iris
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt


     -------------------------------------- 41.8/41.8 kB 335.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ------------------------------------ 201.9/201.9 kB 679.8 kB/s eta 0:00:00
     ---------------------------------------- 8.8/8.8 MB 789.0 kB/s eta 0:00:00
     -------------------------------------- 98.8/98.8 kB 218.1 kB/s eta 0:00:00
     ------------------------------------ 468.5/468.5 kB 212.6 kB/s eta 0:00:00
  Created wheel for factor_analyzer: filename=factor_analyzer-0.4.1-py2.py3-none-any.whl size=42070 sha256=c36a714458667ac7ae64625e8a3a09dd208b03aaed0c5c3918cb922a43542f2b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\6d\32\bd\460a71becd83f7d77152f